In [ ]:
import numpy as np
import sys

In [ ]:
from misc import load, load_class_labels

dataset_root = "./dataset/"

train_dir = dataset_root + "train_set/"
val_dir = dataset_root + "val_set/"
val_degraded_dir = dataset_root + "val_set_degraded/"

sample_rate = 1
train_info = load(dataset_root + 'labels/train.csv', 1, sample_rate)
val_info = load(dataset_root + 'labels/validation.csv')

train_image_names = train_info.iloc[:, 0].values
val_image_names = val_info.iloc[:, 0].values

train_labels = train_info.iloc[:, 1].values
val_labels = val_info.iloc[:, 1].values

class_labels = load_class_labels(dataset_root + 'classes.txt')

In [ ]:
from nn_filter import NN_filter

filter = NN_filter(train_info, train_dir, 'efficient_net', batch_size=25)

In [ ]:
features, labels, image_names = filter.fit_train()

## Extract validation from trainig set

In [ ]:
import pandas as pd
train_info = load(dataset_root + 'train_info_dirty.csv', 1, 0.8)
val_info = load(dataset_root + 'train_info_dirty.csv', 1, 1);


same = val_info['filename'].isin(train_info['filename']);
val_info.drop(val_info[same].index, inplace=True)

#train_info.to_csv(dataset_root + '/labels/train.csv', header=False, index=False);
#val_info.to_csv(dataset_root + '/labels/validation.csv', header=False, index=False);

## Outlier detection using pca on NN features

In [ ]:
features.shape

In [ ]:
pca = filter.fit_pca(features, components=9)

In [ ]:
significance_threshold = 0.8
np.set_printoptions(threshold=sys.maxsize)
cum_prod = filter.analyze_pca_components(pca)

#cum_prod = [pair for pair in cum_prod if pair[1] > significance_threshold][0]
#print('num of component that should be used: ', cum_prod[0])

In [ ]:
new_space = filter.pca_features(pca, features);

In [ ]:
from misc import group_stats
mean, cov = group_stats(new_space, labels)

In [ ]:
from image_discard import find_outliers_per_class
names, distances = find_outliers_per_class(mean, cov, new_space, labels, image_names)

In [ ]:
from image_discard import write_discarded_images
#write_discarded_images(names, class_labels, "./discarded_pca_nn_features/", train_dir)

## Outlier detection with NN features

In [ ]:
from image_discard import write_discarded_images, find_outliers_iter, write_cleaned_csv
names, distances = find_outliers_iter(features, labels, image_names, threshold=12, num_iter=15)
write_discarded_images(names, class_labels, "./discarded_scaling_features/", train_dir)
write_cleaned_csv(train_info, names, dataset_root)

## Outlier detection with KNN

In [ ]:
use_pca = False

In [ ]:
knn = filter.fit_knn(features, labels, use_pca);

In [ ]:
names = filter.filter_with_knn(knn, use_pca);
write_discarded_images(names, class_labels, "./discarded_knn/", train_dir)